In [ ]:
import numpy as np
import cv2
from skimage import io, color
import matplotlib as plt

def imagePrepare(filename):
    '''
    function imagePrepare
    objective: processes image to binary for detection function
    parameters: filename of original microscope image
    returns: image (original color image), nuclei-isolated image (cleaned up), non-cleaned up nuclei-isolated image, L*a*b* color image, and whole epidermis image
    '''
    #reads image via openCV
    image = cv2.imread(str(filename))
    short_file = str(filename.split('.')[0])
    rgbIm = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    labIm = cv2.cvtColor(rgbIm, cv2.COLOR_RGB2Lab)
    #find image area in sq. pixels
    area = image.shape[0] * image.shape[1]
    #calculates sub-region size for adaptive threshold to get 125,000 sub-regions (produces cleanest binarization)
    thresholdpix = int((area/125000)/4)
    if thresholdpix % 2 == 0:
        thresholdpix += 1
    #converts image to greyscale
    grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #separate whole epidermis from background
    ret, epidermis = cv2.threshold(grey, 254, 255, cv2.THRESH_BINARY)
    #applies 5px median blur for smoothing
    blurM = cv2.medianBlur(grey, 5)
    #modified otsu binarization to white and black using sub-region size determined above
    thresh = cv2.adaptiveThreshold(blurM, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, thresholdpix, 6)
    cv2.imwrite(short_file + '_thresh.jpg', thresh)
    #sets object size as circle/ellipse for morphology changes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    #erosion to separate closely packed nuclei
    th2 = cv2.morphologyEx(thresh, cv2.MORPH_ERODE, kernel, iterations = 1)
    #closing -- dilation followed by erosion to clearly define edges
    th2 = cv2.morphologyEx(th2, cv2.MORPH_CLOSE, kernel, iterations = 1)
    cv2.imwrite(short_file + '_th2.jpg', th2)
    shed = th2
    totalnuc = thresh
    return image, shed, totalnuc, labIm, epidermis


def contourFinder(shed, short_file):
    '''
    function contourFinder
    objective: identify nuclei from binary image
    parameters: shed (binary image returned from image processing fxn)
    returns: image with nuclei borders drawn on and an array of border coordinates
    '''
    image = shed
    #creates array of contoured areas in binary image
    cont, hier = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print('number of contours found: '+str(len(cont)))
    #draws found contours onto image
    cv2.drawContours(image, cont, -1, (0,0,255), 2, cv2.LINE_AA)
    mask = np.zeros(shed.shape[:2], dtype = 'uint8')
    image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    #loops through contour points to refine outline of contours and draw onto image
    for i, contours in enumerate(cont):
        for j, contpoint in enumerate(contours):
            cv2.circle(image, ((contpoint[0][0], contpoint[0][1])), 2, (0,255,0), 2, cv2.LINE_AA)
    #writes number of detections onto image
    image = cv2.putText(image, str('number of detections: '+str(len(cont))), (50, 1000), cv2.FONT_HERSHEY_SIMPLEX, 4, (0,0,255), 5, cv2.LINE_AA)
    #outputs image
    cv2.imwrite(short_file+'_contoured.jpg', image)
    return image, cont


def cytoColor(epi, totalnuc, lab, short_file):
    '''
    function cytoColor
    objective: determine the average red/blue color ratio of the cytoplasm to compare to nuclei
    parameters: binary image of whole epidermis, binary image of nuclei only, original color image in L*a*b* color
    returns: average red/blue ratio of cytoplasm
    '''
    #subtraction of binary epidermis image and binary nuclei image to isolate cytoplasm only
    inv = np.invert(epi)
    bg = np.nonzero(inv)
    nuc = np.nonzero(totalnuc)
    cyto = np.subtract(inv, totalnuc)
    cv2.imwrite(short_file+'_bgminusnuc.jpg', cyto)
    #calculate average color in L*a*b* color space
    mean = cv2.mean(lab, mask=cyto)
    #convert to conventional L*a*b* color scales instead of 0-255
    l = float(mean[0])/2.55
    a = float(mean[1]) - 128
    b = float(mean[2]) - 128
    #find ratio of red to blue (indicator of innate tissue brownness due to variability between samples)
    if a != 0 and b != 0:
        ab = abs(a/b)
    elif a == 0:
        ab = abs(b/2)
    else:
        ab = abs(a/2)
    print(ab)
    return ab
    

def maskMaker(bwim, image, c):
    '''
    function maskMaker
    objective: creates separate binary masks for each identified nucleus
    parameters: binary image and array of contour points
    returns: average color inside the boundary of a nucleus contour
    '''
    #catches empty or open contours
    k = 0
    coordList = []
    #fills in contour 
    mask = np.zeros(image.shape[:2], dtype = 'uint8')
    cv2.drawContours(mask, [c], -1, 255, -1)
    mask = cv2.erode(mask, None, iterations = 2)
    #find pixel coordinates of all points inside contour instead of just contour boundary
    if cv2.findNonZero(mask) is not None:
        coord = cv2.findNonZero(mask)
        size = cv2.countNonZero(mask)
        for i in range(len(coord)):
            coordList.append(coord[i])
    else:
        size = 0
        k += 1
    return k, coordList, size

def nucSize(contours, shed, lab):
    sizeArr = []
    for c in contours:
        k, coordList, size = maskMaker(shed, lab, c)
        if size != 0:
            print(size)
            sizeArr.append(size)
    sizeArr.sort()
    mid = len(sizeArr) // 2
    median = (sizeArr[mid]+sizeArr[~mid])/2
    firstquart = np.quantile(sizeArr, 0.15)
    thirdquart = np.quantile(sizeArr, 0.95)
    print('median: ' + str(median))
    print('average: ' + str((sum(sizeArr)/len(sizeArr))))
    print('10th quartile: '+str(firstquart))
    print('95th quartile:'+str(thirdquart))
    return firstquart, thirdquart


def colorCalc(contours, lab, image, shed, cyto, short_file):
    '''
    function colorCalc
    objective: determine the location and amount of positive and negative nuclei
    parameters: array of contour coordinates, color image converted into L*a*b* color space
    returns: image with positive and negative nuclei identified and counted
    '''
    sizeMin, sizeMax = nucSize(contours, shed, lab)
    #adjusts red/blue threshold in case of overly blue image
    if cyto < 1:
        t = cyto
    else:
        t = cyto
    #initiate counters for iteration through contours
    posNuc = 0
    negNuc = 0
    contourCount = len(contours)
    print('height lab: '+ str(lab.shape[0]))
    print('width lab: ' + str(lab.shape[1]))
    #iterates through contours
    for c in contours:
        #identifying centroid for outlining on image later
        M = cv2.moments(c)
        if int(M['m00']) != 0:
            cX = int(M["m10"]/M['m00'])
            cY = int(M['m01']/M['m00'])
        else:
            cX = 0
            cY = 0
        #fills in mask for current contour
        k, coordList, size = maskMaker(shed, lab, c)
        c = c.astype('float')
        c = c.astype('int')
        #nucleus size constraint
        if size > sizeMin and size < sizeMax:
            #initiates counts for iterating through pixels
            posPix = 0
            negPix = 0
            totalPix = 0
            for j in coordList:
                dst= cv2.transpose(j)
                #identifies L*a*b* color and converts to usual scale
                color = lab[dst[1],dst[0]][0]
                colorList = []
                for i in range(len(color)):
                    if i == 0:
                        colorList.append(float(color[0]/2.55))
                    else:
                        colorList.append(float(color[i]-128))
                color = colorList
                #catch divide by 0 errors when red-green value or blue-yellow value is in exact middle of spectrum
                #check if pixel meets red/blue ratio threshold to be considered blue vs. brown
                if color[1] != 0 and color[2] != 0:
                    if abs(color[1]/color[2]) >= t and color[2] > -20:
                        posPix += 1
                    else:
                        negPix += 1
                elif color[1] == 0 and color[2] != 0:
                    if abs(color[2]/2) >= t and color[2] > -20:
                        posPix += 1
                    else:
                        negPix += 1
                elif color[1] == 0 and color[2] == 0:
                    negPix += 1
                else:
                    if abs(color[1]/2) >= t:
                        posPix += 1
                    else:
                        negPix += 1
                totalPix = posPix + negPix
            #if 40% of pixels are brown the nucleus is considered brown (positive)
            #draws red (positive) or blue (negative) outline around each nucleus
            if posPix/totalPix >= 0.33:
                posNuc += 1
                cv2.drawContours(image, [c], -1, (0,0,255), 2)
                cv2.putText(image, 'p', (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0),2)
            else:
                negNuc += 1
                cv2.drawContours(image, [c], -1, (255,0,0), 2)
                cv2.putText(image, 'n', (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),2)
        else:
            contourCount -= 1
    cv2.imwrite(short_file + '_final.jpg', image)
    #output counts for total quantification
    print('total nuclei: ' + str(contourCount) + ' positive nuclei: ' + str(posNuc) + ' negative nuclei: ' +str(negNuc))      
    return posNuc+negNuc

        
def runTotal(filename):
    short_file = str(filename.split('.')[0])
    image, shed, totalnuc, lab, epi = imagePrepare(filename)
    contImage, contours = contourFinder(shed, short_file)
    cyto = cytoColor(epi, totalnuc, lab, short_file)
    colorCalc(contours, lab, image, shed, cyto, short_file)
    
runTotal('your_filename.tif')